In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from opencluster.utils import prepare_and_validate_stars
from opencluster.cmd import interactive_CMD


# How to use this:

1. Choose the `C` variable to be one of the clusters. 
2. The code in the next cell will drop all targets without LCs, do the extinction correction, transformation of photometric systems, etc., i.e. all that is implemented in `utils.prepare_and_validate_stars`.
2. Generate CMDs for all colors used in CTRs later\*. Open the `.hmtl` file in a browser (Firefox and Chrome both may work).
3. Select data points that fall off the MS most obviously.
5. Click the _Save_ button and save to `cmd/outliers/<cluster>_<band1>_<band2>.txt`.
6. Go back to the notebook and read in all outlier files, add the corresponding flags to the `todrop` column.
7. Write out the resulting table to `cmd/cmd_flagged/<cluster>_cmd.csv`


In [0]:
#Cycle through all cluster names here
C = 'praesepe'

In [37]:

if C!= "hyades":
    
    df = pd.read_csv('input/StarHorse_output/{}_topcat_w_extinction_percentiles.csv'.format(C))
    if C in ["pleiades","praesepe"]:
        extinction = False
    else:
        extinction = True
else:
    df = pd.read_csv('input/StarHorse_output/hyades_topcat_w_extinction_percentiles.csv')
    df['eup_Ext_BPRP_Gaia'] = np.nan
    df['elo_Ext_BPRP_Gaia'] = np.nan
    extinction = False
    
if C == 'ngc2682':
    df = df[(~df["RAJ2000_N"].isnull() | ~df["RAJ2000_K2"].isnull())]
    df.EPIC = df.EPIC.fillna(df.id_N).astype(int)
elif C == 'ngc2168':
    df = df[(~df["RAJ2000_SF"].isnull() | ~df["RAJ2000_K2"].isnull())]
    df.EPIC = df.EPIC.fillna(df['2MASS_id_SF']).astype(int)
else:
    df = df[~df["RAJ2000_K2"].isnull()]
    
df = prepare_and_validate_stars(df, extinction=extinction)

In [0]:
colors = [('g_SDSS','J_2MASS'), ('BP_Gaia','RP_Gaia'), ('g_SDSS','z_SDSS'),
                       ('g_SDSS','i_SDSS'), ('g_SDSS','H_2MASS'), ('g_SDSS','K_2MASS'),
                       ('r_SDSS','z_SDSS'), ('r_SDSS','J_2MASS'),]

In [18]:
interactive_CMD(df,cid1=colors[0][0],cid2=colors[0][1])

In [19]:
interactive_CMD(df,cid1=colors[1][0],cid2=colors[1][1])

In [20]:
interactive_CMD(df,cid1=colors[2][0],cid2=colors[2][1])

In [21]:
interactive_CMD(df,cid1=colors[3][0],cid2=colors[3][1])

In [22]:
interactive_CMD(df,cid1=colors[4][0],cid2=colors[4][1])

In [23]:
interactive_CMD(df,cid1=colors[5][0],cid2=colors[5][1])

In [24]:
interactive_CMD(df,cid1=colors[6][0],cid2=colors[6][1])

In [25]:
interactive_CMD(df,cid1=colors[7][0],cid2=colors[7][1])

# Now collect all outliers and flag them

In [38]:
for (i,j) in colors:
    c1, c2 = i.split('_')[0],j.split('_')[0]
    file = 'cmd/outliers/{}_{}_{}.txt'.format(C,c1,c2)
    if os.stat(file).st_size != 0:
        a = pd.read_csv(file,header=None,sep=' ').T.dropna(how='any')
        a = a.rename(index=str, columns={0:'outlier'}).astype(int)
        def add_flag(s):
            cond = isinstance(s, str)
            sadd = '{}-{} outlier'.format(c1,c2)
            if cond:
                print(s)
                return '{}, {}'.format(s,sadd)
            else:
                print(sadd)
                return sadd
        df.loc[df.EPIC.isin(a.outlier),'todrop'] =  df.loc[df.EPIC.isin(a.outlier),'todrop'].map(add_flag)

In [39]:
df.groupby("todrop").todrop.count()

In [40]:
df.to_csv('cmd/cmd_flagged/{}_cmd.csv'.format(C))